### Adjust Word Group In Youtube Sentence Word Usage Analysis

In [1]:
import os
import multiprocessing
#import multiprocessing as mp
from multiprocessing import Process, Manager, Pool, Queue
from itertools import islice
from collections import Counter
import re
import pandas as pd
import numpy as np
import glob
import nltk
from nltk import word_tokenize
from functools import reduce
from pathlib import Path
import shutil

In [2]:
nprocs = multiprocessing.cpu_count()
print(f"Number of CPU cores: {nprocs}")

Number of CPU cores: 16


In [3]:
# language pair
lang_folder = "Turkish"  # Arabic, English, French, German, Turkish, Spanish, Portuguese, Dutch, Italian ==> target language for learner
#lang_pair = "English"  # Arabic, English, French, German, Turkish, Spanish, Portuguese, Dutch, Italian ==> native language

# native word select for Part 2 
word_start = 0  # 0  # native word start index
word_end = 1000 # 200 # native word end index

# word all usage in sent
word_use_num_max = 7  # 5
word_use_num_min = 1  # 1

# youtube sentence usage for remain word 
youtube_sent_usage = "selected"  # "all","selected"

In [4]:
path = f"/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/{lang_folder.capitalize()}/\
Deployment2/Result/3-Adjust Word Group In Youtube Sentence Word Usage Analysis/{lang_folder.capitalize()}"

Path(path).mkdir(parents=True, exist_ok=True)

In [5]:
def word_in_wordgroup_simple(source_word_list, df_target, target_column, word_sample_num):

    '''word_in_wordgroup_simple(not_in_sent_word_list, df_youtube_sent_select, "search_string", 5)\n
       source_word_list is searching word list\n
       df_target is dataframe, target_column are dataframe column string name\n
       word_sample_num is searching sample number. 
    '''
    
    df_select = df_target[[f"{target_column}"]].dropna()
    df_result = pd.DataFrame()
    for i in source_word_list:
        try:
            word_in_word_cluster = df_select[df_select[f"{target_column}"].str.contains(fr"(?:\s|^){i}(?:\s|$)", na=True)].head(word_sample_num)    
        except:
            pass        
        word_in_word_cluster.insert(0,"word",i)
        df_result = pd.concat([df_result,word_in_word_cluster], axis=0)
    df_result.reset_index(drop=True, inplace=True)

    return df_result

#### Word Usage

In [6]:
def word_index(df_word, word_column, sentence):
    word_index_list = []
    word_index_dict = {var2:var1 for (var1, var2) in enumerate(df_word[f"{word_column}"])}
    words = word_tokenize(sentence)
    for word in words:
        value = word_index_dict[f"{word}"]
        word_index_list.append(value)
    return word_index_list

In [7]:
df_word_all = pd.read_excel(f"/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/{lang_folder.lower().capitalize()}/Deployment/Data/Word/Word_Merge_Preprocess.xlsx")
df_word_all

,word,frequency
0,bir,18835735
1,bu,11062659
2,ne,8025880
3,ve,7766036
4,için,5484109
...,...,...
987924,karneleme,5
987925,karnaya,5
987926,dörtlümüzün,5
987927,karnavalınız,5


In [8]:
df_word_select = df_word_all.iloc[word_start:word_end,]
df_word_select.reset_index(inplace=True)
df_word_select

,index,word,frequency
0,0,bir,18835735
1,1,bu,11062659
2,2,ne,8025880
3,3,ve,7766036
4,4,için,5484109
...,...,...,...
995,995,getirdim,70790
996,996,dinleyin,70632
997,997,hale,70625
998,998,kavga,70611


In [9]:
df_youtube_sent_result = pd.read_excel(f"/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/{lang_folder.lower().capitalize()}/Deployment2/Result/\
2-Word Group In Youtube Sentence/{lang_folder.lower().capitalize()}/{lang_folder.lower().capitalize()}_Word_Group_Max_In_Youtube_Sentence_{word_end}_Word6.xlsx")
df_youtube_sent_result

,search_string,start_time,end_time,sentence,video_id,video_url
0,başka biriyle herhangi bir şey yaşıyor musun k...,319,325,başka biriyle herhangi bir şey yaşıyor musun k...,KsZ_8oIQsv4,https://www.youtube.com/watch?v=KsZ_8oIQsv4&t=...
1,gidelim bir bakalım hep birlikte bakalım bir o...,987,988,yerleştirmeye yumurtasını seviyoruz çok güzel ...,eqkDognLtTc,https://www.youtube.com/watch?v=eqkDognLtTc&t=...
2,nasıl olduğunu gördün zaten artık değil mi tek...,2614,2620,nasıl olduğunu gördün zaten artık değil mi tek...,N4YDaU51lvI,https://www.youtube.com/watch?v=N4YDaU51lvI&t=...
3,benim kızım tamam bir şey yapmış hata yapmış o...,3090,3097,benim kızım tamam bir şey yapmış hata yapmış o...,CIQ3Ave21Kw,https://www.youtube.com/watch?v=CIQ3Ave21Kw&t=...
4,bir şekilde söylüyorum ve o zaman eminim ki ço...,4774,4781,bir şekilde söylüyorum ve o zaman eminim ki ço...,cUAQrHbbpZ8,https://www.youtube.com/watch?v=cUAQrHbbpZ8&t=...
...,...,...,...,...,...,...
593015,a de a a,3170,3173,a de a a,X7D_dmahSD0,NaN
593016,a a i ya,602,603,a a i ya,ZijHforW6q4,NaN
593017,ol o o o,2258,2260,abone ol o o o,K0b_9Xzy79A,NaN
593018,ol o o o,233,237,hoşça kalın allaha emanet ol abone ol o o o,JfhX1WDJjks,NaN


In [10]:
df_youtube_sent_result["word_index_list"] = df_youtube_sent_result["search_string"].apply(lambda sent: word_index(df_word_select, "word", sent))
df_youtube_sent_result

,search_string,start_time,end_time,sentence,video_id,video_url,word_index_list
0,başka biriyle herhangi bir şey yaşıyor musun k...,319,325,başka biriyle herhangi bir şey yaşıyor musun k...,KsZ_8oIQsv4,https://www.youtube.com/watch?v=KsZ_8oIQsv4&t=...,"[71, 871, 467, 0, 16, 888, 53, 247, 97, 4, 24,..."
1,gidelim bir bakalım hep birlikte bakalım bir o...,987,988,yerleştirmeye yumurtasını seviyoruz çok güzel ...,eqkDognLtTc,https://www.youtube.com/watch?v=eqkDognLtTc&t=...,"[204, 0, 175, 208, 186, 175, 0, 93, 75, 516, 4..."
2,nasıl olduğunu gördün zaten artık değil mi tek...,2614,2620,nasıl olduğunu gördün zaten artık değil mi tek...,N4YDaU51lvI,https://www.youtube.com/watch?v=N4YDaU51lvI&t=...,"[39, 40, 331, 192, 61, 14, 5, 160, 0, 495, 644..."
3,benim kızım tamam bir şey yapmış hata yapmış o...,3090,3097,benim kızım tamam bir şey yapmış hata yapmış o...,CIQ3Ave21Kw,https://www.youtube.com/watch?v=CIQ3Ave21Kw&t=...,"[31, 745, 29, 0, 16, 962, 692, 962, 129, 7, 26..."
4,bir şekilde söylüyorum ve o zaman eminim ki ço...,4774,4781,bir şekilde söylüyorum ve o zaman eminim ki ço...,cUAQrHbbpZ8,https://www.youtube.com/watch?v=cUAQrHbbpZ8&t=...,"[0, 181, 769, 3, 6, 35, 294, 33, 896, 249, 31,..."
...,...,...,...,...,...,...,...
593015,a de a a,3170,3173,a de a a,X7D_dmahSD0,NaN,"[574, 8, 574, 574]"
593016,a a i ya,602,603,a a i ya,ZijHforW6q4,NaN,"[574, 574, 775, 45]"
593017,ol o o o,2258,2260,abone ol o o o,K0b_9Xzy79A,NaN,"[118, 6, 6, 6]"
593018,ol o o o,233,237,hoşça kalın allaha emanet ol abone ol o o o,JfhX1WDJjks,NaN,"[118, 6, 6, 6]"


In [11]:
#df_youtube_sent_select.to_excel(f"{lang_folder.capitalize()}_{word_end}_Word_Group_In_Youtube_Sentence_Word_Index_List.xlsx", index=False)

In [12]:
df_youtube_sent_select = df_youtube_sent_result[df_youtube_sent_result["start_time"] <= 700]
df_youtube_sent_select.reset_index(drop=True, inplace=True)
df_youtube_sent_select

,search_string,start_time,end_time,sentence,video_id,video_url,word_index_list
0,başka biriyle herhangi bir şey yaşıyor musun k...,319,325,başka biriyle herhangi bir şey yaşıyor musun k...,KsZ_8oIQsv4,https://www.youtube.com/watch?v=KsZ_8oIQsv4&t=...,"[71, 871, 467, 0, 16, 888, 53, 247, 97, 4, 24,..."
1,sıcak mı evet böyle işte tamam bir şey yok ora...,603,612,sıcak mı evet böyle işte tamam bir şey yok ora...,Mh3bgDLKt2o,https://www.youtube.com/watch?v=Mh3bgDLKt2o&t=...,"[723, 13, 12, 56, 108, 29, 0, 16, 22, 93, 175,..."
2,benim burada ne işim var neden bunu yapıyorum ...,273,278,benim burada ne işim var neden bunu yapıyorum ...,sHnvEDmE3so,https://www.youtube.com/watch?v=sHnvEDmE3so&t=...,"[31, 38, 2, 844, 11, 34, 26, 662, 61, 218, 20,..."
3,bırakın falan diyor tamam tahmin ettim zaten ç...,350,356,uygulamayı bırakın falan diyor tamam tahmin et...,cmQjfzOKFII,https://www.youtube.com/watch?v=cmQjfzOKFII&t=...,"[589, 268, 716, 29, 612, 360, 192, 9, 0, 99, 4..."
4,belli değil benim ne yapacağım belli değil ne ...,93,99,kaderim belli değil benim ne yapacağım belli d...,VBdcD7hLGuo,https://www.youtube.com/watch?v=VBdcD7hLGuo&t=93s,"[634, 14, 31, 2, 544, 634, 14, 2, 83, 634, 14,..."
...,...,...,...,...,...,...,...
181752,ol o o ne,633,636,abone ol o o ne arabası bu arkadaşlar zaten ka...,0Y065FGH_T0,NaN,"[118, 6, 6, 2]"
181753,ol o o i,606,609,abone ol o o i yi akşamlar bilmediğim için abo...,JyzAOR0GTs4,NaN,"[118, 6, 6, 775]"
181754,mi o o o,419,428,mi o o o,5ofdGcvaeZU,NaN,"[5, 6, 6, 6]"
181755,a a i ya,602,603,a a i ya,ZijHforW6q4,NaN,"[574, 574, 775, 45]"


In [13]:
# # word usage in max condition
# result_list = []
# index_list = []
# for i in range(len(df_youtube_sent_select)):
#     search = df_youtube_sent_select.loc[i,"search_string"]
#     start=df_youtube_sent_select.loc[i,"start_time"]
#     end=df_youtube_sent_select.loc[i,"end_time"]
#     sent=df_youtube_sent_select.loc[i,"sentence"]
#     id = df_youtube_sent_select.loc[i,"video_id"]
#     id_url = df_youtube_sent_select.loc[i,"video_url"]
#     word_index = df_youtube_sent_select.loc[i,"word_index_list"]
#     list_var = index_list + word_index
#     #list_var = index_list + [x for x in word_index]
# 
#     dict_list_count = Counter(list_var)
#     count_list = list(dict_list_count.values())
# 
#     if any([True if i>word_use_num_max else False for i in count_list]):
#         index_list = index_list
#     else:
#         index_list = index_list + word_index
#         result_list.append([search,start,end,sent,id,id_url,word_index])
#     

In [14]:
# word usage in min max condition
word_num_index_dict = {}
for i in df_word_select["index"]:
    word_num_index_dict[i] = 0

result_list = []
index_list = []
for i in range(len(df_youtube_sent_select)):
    search = df_youtube_sent_select.loc[i,"search_string"]
    start=df_youtube_sent_select.loc[i,"start_time"]
    end=df_youtube_sent_select.loc[i,"end_time"]
    sent=df_youtube_sent_select.loc[i,"sentence"]
    id = df_youtube_sent_select.loc[i,"video_id"]
    id_url = df_youtube_sent_select.loc[i,"video_url"]
    word_index = df_youtube_sent_select.loc[i,"word_index_list"]
    list_var = index_list + word_index
    #list_var = index_list + [x for x in word_index]
    # word count for max
    dict_list_count = Counter(list_var)
    count_list = list(dict_list_count.values())
     # word count for min
    count_list2 = list(word_num_index_dict.values())   

    if any([True if i>word_use_num_max else False for i in count_list]) or not(any([True if j<word_use_num_min else False for j in count_list2])):
        index_list = index_list
    else:
        index_list = index_list + word_index
        result_list.append([search,start,end,sent,id,id_url,word_index])

        for item2 in dict_list_count.items(): 
            word_num_index_dict[item2[0]] = item2[1]

print(f"Max condition: {any([True if i>word_use_num_max else False for i in count_list2])} \nMin condition: {any([True if j<word_use_num_min else False for j in count_list2])}") 

Max condition: False 
Min condition: True


In [15]:
#word_num_index_dict

In [16]:
set_index = set(index_list)
set_all_index = set([x for x in df_word_select["index"]])

In [17]:
not_in_sent = set_all_index.difference(set_index)
not_in_sent = list(not_in_sent)
not_in_sent

[528,
 534,
 541,
 547,
 561,
 566,
 572,
 574,
 575,
 576,
 577,
 581,
 595,
 599,
 606,
 615,
 622,
 115,
 628,
 632,
 122,
 636,
 639,
 646,
 658,
 659,
 661,
 668,
 671,
 685,
 686,
 709,
 200,
 201,
 724,
 214,
 726,
 729,
 736,
 737,
 738,
 227,
 746,
 756,
 762,
 765,
 770,
 772,
 774,
 778,
 790,
 791,
 793,
 795,
 286,
 800,
 805,
 294,
 806,
 810,
 815,
 827,
 319,
 832,
 322,
 323,
 839,
 841,
 843,
 846,
 848,
 337,
 852,
 341,
 342,
 856,
 346,
 858,
 862,
 863,
 352,
 864,
 865,
 867,
 872,
 362,
 875,
 876,
 366,
 878,
 886,
 898,
 899,
 900,
 902,
 392,
 914,
 915,
 404,
 920,
 922,
 923,
 924,
 925,
 926,
 927,
 931,
 934,
 936,
 944,
 950,
 952,
 955,
 956,
 445,
 448,
 960,
 450,
 963,
 454,
 460,
 972,
 973,
 976,
 977,
 981,
 472,
 987,
 988,
 991,
 494,
 500,
 503]

In [18]:
#Counter(index_list)

In [19]:
df_youtube_sent_sample = pd.DataFrame(result_list, columns=["search_string","start_time","end_time","sentence","video_id","video_url","word_index"])
df_youtube_sent_sample

,search_string,start_time,end_time,sentence,video_id,video_url,word_index
0,başka biriyle herhangi bir şey yaşıyor musun k...,319,325,başka biriyle herhangi bir şey yaşıyor musun k...,KsZ_8oIQsv4,https://www.youtube.com/watch?v=KsZ_8oIQsv4&t=...,"[71, 871, 467, 0, 16, 888, 53, 247, 97, 4, 24,..."
1,sıcak mı evet böyle işte tamam bir şey yok ora...,603,612,sıcak mı evet böyle işte tamam bir şey yok ora...,Mh3bgDLKt2o,https://www.youtube.com/watch?v=Mh3bgDLKt2o&t=...,"[723, 13, 12, 56, 108, 29, 0, 16, 22, 93, 175,..."
2,benim burada ne işim var neden bunu yapıyorum ...,273,278,benim burada ne işim var neden bunu yapıyorum ...,sHnvEDmE3so,https://www.youtube.com/watch?v=sHnvEDmE3so&t=...,"[31, 38, 2, 844, 11, 34, 26, 662, 61, 218, 20,..."
3,bırakın falan diyor tamam tahmin ettim zaten ç...,350,356,uygulamayı bırakın falan diyor tamam tahmin et...,cmQjfzOKFII,https://www.youtube.com/watch?v=cmQjfzOKFII&t=...,"[589, 268, 716, 29, 612, 360, 192, 9, 0, 99, 4..."
4,belli değil benim ne yapacağım belli değil ne ...,93,99,kaderim belli değil benim ne yapacağım belli d...,VBdcD7hLGuo,https://www.youtube.com/watch?v=VBdcD7hLGuo&t=93s,"[634, 14, 31, 2, 544, 634, 14, 2, 83, 634, 14,..."
...,...,...,...,...,...,...,...
670,ara ara ara sıra,315,320,oluşan köpükleri yine alıyorum ara ara ara sır...,uFwJxlTZEKs,NaN,"[442, 442, 442, 883]"
671,kız gördün mü az,543,545,aa kız gördün mü az kalsın unutuyordum,ys_HDCt6xEY,NaN,"[138, 331, 163, 230]"
672,su alıp ara sıra,451,453,de bir miktar su alıp ara sıra elimizi ıslatıp...,Ywi4siRg364,NaN,"[355, 711, 442, 883]"
673,ah ah ah oh oh,318,320,ah ah ah oh oh,ZRleefh0Z8Y,NaN,"[315, 315, 315, 187, 187]"


In [20]:
df_youtube_sent_sample.to_excel(f"{lang_folder.capitalize()}_{word_end}_Word_Group_In_Youtube_Sentence_Word_Index_List_Sample.xlsx", index=False)

##### Selected Result Word Count

In [21]:
#df_selected_file = pd.read_excel(f"/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/{lang_folder.capitalize()}/Deployment2/Result/6-Youtube Sentence Pos Tag Analysis/Deploy Result Manuel/{lang_folder.capitalize()}_200_Word_Group_In_Youtube_Sentence_Word_Index_List_Sample Revised.xlsx")
df_selected_file = pd.read_excel(f"{lang_folder.capitalize()}_{word_end}_Word_Group_In_Youtube_Sentence_Word_Index_List_Sample.xlsx")
df_selected_file

,search_string,start_time,end_time,sentence,video_id,video_url,word_index
0,başka biriyle herhangi bir şey yaşıyor musun k...,319,325,başka biriyle herhangi bir şey yaşıyor musun k...,KsZ_8oIQsv4,https://www.youtube.com/watch?v=KsZ_8oIQsv4&t=...,"[71, 871, 467, 0, 16, 888, 53, 247, 97, 4, 24,..."
1,sıcak mı evet böyle işte tamam bir şey yok ora...,603,612,sıcak mı evet böyle işte tamam bir şey yok ora...,Mh3bgDLKt2o,https://www.youtube.com/watch?v=Mh3bgDLKt2o&t=...,"[723, 13, 12, 56, 108, 29, 0, 16, 22, 93, 175,..."
2,benim burada ne işim var neden bunu yapıyorum ...,273,278,benim burada ne işim var neden bunu yapıyorum ...,sHnvEDmE3so,https://www.youtube.com/watch?v=sHnvEDmE3so&t=...,"[31, 38, 2, 844, 11, 34, 26, 662, 61, 218, 20,..."
3,bırakın falan diyor tamam tahmin ettim zaten ç...,350,356,uygulamayı bırakın falan diyor tamam tahmin et...,cmQjfzOKFII,https://www.youtube.com/watch?v=cmQjfzOKFII&t=...,"[589, 268, 716, 29, 612, 360, 192, 9, 0, 99, 4..."
4,belli değil benim ne yapacağım belli değil ne ...,93,99,kaderim belli değil benim ne yapacağım belli d...,VBdcD7hLGuo,https://www.youtube.com/watch?v=VBdcD7hLGuo&t=93s,"[634, 14, 31, 2, 544, 634, 14, 2, 83, 634, 14,..."
...,...,...,...,...,...,...,...
670,ara ara ara sıra,315,320,oluşan köpükleri yine alıyorum ara ara ara sır...,uFwJxlTZEKs,NaN,"[442, 442, 442, 883]"
671,kız gördün mü az,543,545,aa kız gördün mü az kalsın unutuyordum,ys_HDCt6xEY,NaN,"[138, 331, 163, 230]"
672,su alıp ara sıra,451,453,de bir miktar su alıp ara sıra elimizi ıslatıp...,Ywi4siRg364,NaN,"[355, 711, 442, 883]"
673,ah ah ah oh oh,318,320,ah ah ah oh oh,ZRleefh0Z8Y,NaN,"[315, 315, 315, 187, 187]"


In [22]:
def word_count_result(df,column_list): # df is dataframe, column_list is list value
    '''
    word_count_bool(df, column_list): df columns word count for word frequency\n
    df is dataframe, column_list is list value\n
    word_count_bool(df, ["word","twogram"]):
    '''
    list_all = []
    for i in df.loc[:,[x for x in column_list]].columns:
        var_list = df[f"{i}"].dropna().tolist()
        for j in var_list:
            list_all.append(j)
    text = " ".join(list_all)
    word_list = re.findall(r"\w+",text, re.UNICODE)
    df_word_list = pd.DataFrame(word_list, columns=["word"])
    #df_word_list.rename(columns={0:"word"}, inplace=True)
    df_word_count = pd.DataFrame(df_word_list.value_counts())
    df_word_count.reset_index(inplace=True)
    df_word_count.rename(columns={0:"word_count"}, inplace=True)
    
    return  df_word_count

In [23]:
df_shared_word_manuel_count = word_count_result(df_selected_file, df_selected_file.columns[0:1])
df_shared_word_manuel_count

,word,word_count
0,içinde,7
1,iyi,7
2,kendini,7
3,kendine,7
4,kendi,7
...,...,...
862,bilirsin,1
863,dinleyin,1
864,güçlü,1
865,oldukça,1


In [24]:
df_shared_word_manuel_count.to_excel(f"{lang_folder.capitalize()}_{word_end}_Word_Group_In_Youtube_Sentence_Word_Index_Word_Count.xlsx", index=False)

##### Not In Sent And Selected Word List

In [25]:
lang_folder = "Turkish"
df_word_all = pd.read_excel(f"/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/{lang_folder.lower().capitalize()}/Deployment/Data/Word/Word_Merge_Preprocess.xlsx")
df_word_all

,word,frequency
0,bir,18835735
1,bu,11062659
2,ne,8025880
3,ve,7766036
4,için,5484109
...,...,...
987924,karneleme,5
987925,karnaya,5
987926,dörtlümüzün,5
987927,karnavalınız,5


In [26]:
#word_start = 0
#word_end = 200
df_word_select = df_word_all.iloc[word_start:word_end,]
df_word_select

,word,frequency
0,bir,18835735
1,bu,11062659
2,ne,8025880
3,ve,7766036
4,için,5484109
...,...,...
995,getirdim,70790
996,dinleyin,70632
997,hale,70625
998,kavga,70611


In [27]:
def index_to_word(index_list, df_word, word_column, reverse=True):
    '''
    index_to_word(not_in_sent, df_word_select, "word", reverse=True)\n
    not in sent is index list as numeric\n
    df_word_select is a dataframe that includes word colunm ("word" is a column)\n
    reverse is used for string index list convert numeric.
    '''
    if reverse:
        word_index_dict = {var1:var2 for (var1, var2) in enumerate(df_word[f"{word_column}"])}
    else:
        word_index_dict = {var2:var1 for (var1, var2) in enumerate(df_word[f"{word_column}"])}

    convert_word_list = []
    for index in index_list:
        convert_word = word_index_dict[index]
        convert_word_list.append(convert_word)
    return convert_word_list        

In [28]:
not_in_sent_word = index_to_word(not_in_sent, df_word_select, "word", reverse=True)
not_in_sent_word

['new',
 'beyler',
 'haklısın',
 'yolunda',
 'çeviri',
 'nasılsın',
 'çeneni',
 'a',
 'geceler',
 'ölüm',
 'tuhaf',
 'tür',
 'mısınız',
 'sevindim',
 'sanıyordum',
 'yapamam',
 'kapa',
 'harika',
 'başkan',
 'güvenlik',
 'üzgünüm',
 's',
 'yapacak',
 'anlaşıldı',
 'sefer',
 'berbat',
 'the',
 'ettin',
 'güç',
 'pardon',
 'frank',
 'bayım',
 'bay',
 'bayan',
 'koca',
 'söyledi',
 'bilmiyor',
 'hak',
 'yapacaksın',
 'adını',
 'tıpkı',
 'olmalı',
 'vakit',
 'olmalısın',
 'temiz',
 'olası',
 'ajan',
 'bulmak',
 'mesaj',
 'buldun',
 'bilir',
 'gitmem',
 'seks',
 'inanamıyorum',
 'hl',
 'sağol',
 'kimsin',
 'eminim',
 'güvenli',
 'bekliyor',
 'cinayet',
 'görünüşe',
 'pekl',
 'içki',
 'kahretsin',
 'umarım',
 'baksana',
 'arkadaşın',
 'film',
 'sebep',
 'şeyden',
 'sence',
 'bağlı',
 'tanrı',
 'ateş',
 'değiliz',
 'yap',
 'sebebi',
 'parçası',
 'sıkı',
 'dr',
 'pislik',
 'orospu',
 'herşey',
 'umurumda',
 'olduğumu',
 'bahsediyorsun',
 'saçma',
 'dolar',
 'oda',
 'eğlenceli',
 'burda',
 'ded

In [29]:
set_not_in_sent_word = set(not_in_sent_word)

In [30]:
set_word = set(df_word_select["word"])
set_select_word = set(df_shared_word_manuel_count["word"])

In [31]:
set_not_in_select = set_word.difference(set_select_word)
set_not_in_select

{'a',
 'adamım',
 'adını',
 'aferin',
 'ahbap',
 'ajan',
 'alır',
 'anlamı',
 'anlaşıldı',
 'arkadaşın',
 'ateş',
 'aşkına',
 'bahsediyorsun',
 'baksana',
 'bay',
 'bayan',
 'bayanlar',
 'bayım',
 'bağlı',
 'başkan',
 'bekliyor',
 'berbat',
 'beyler',
 'bilir',
 'bilmiyor',
 'bok',
 'buldun',
 'bulmak',
 'burda',
 'canına',
 'cinayet',
 'dedektif',
 'değiliz',
 'dolar',
 'dr',
 'düşünüyorsun',
 'eminim',
 'ettin',
 'eğlenceli',
 'fikrim',
 'film',
 'frank',
 'geceler',
 'general',
 'gerekiyordu',
 'gitmem',
 'gurur',
 'görmeye',
 'görünüyor',
 'görünüşe',
 'göster',
 'güvenli',
 'güvenlik',
 'güç',
 'hak',
 'haklısın',
 'harika',
 'hatırlıyorum',
 'hayat',
 'hepiniz',
 'herşey',
 'hl',
 'inanamıyorum',
 'istiyoruz',
 'içki',
 'kahretsin',
 'kahrolası',
 'kan',
 'kapa',
 'kimsin',
 'koca',
 'kral',
 'mesaj',
 'mükemmel',
 'mısınız',
 'nasılsın',
 'new',
 'oda',
 'olası',
 'olduğumu',
 'olmalı',
 'olmalısın',
 'orospu',
 'pardon',
 'parçası',
 'pekl',
 'pislik',
 's',
 'sanmıyorum',
 'sa

In [32]:
not_in_sent_selected_word_list = list(set_not_in_select.union(set_not_in_sent_word))
not_in_sent_selected_word_list

['fikrim',
 'kimsin',
 'serbest',
 'vakit',
 'mesaj',
 'söyledi',
 'içki',
 'kan',
 'geceler',
 'anlaşıldı',
 'hayat',
 'eğlenceli',
 'seks',
 'aşkına',
 'şimdiye',
 'hatırlıyorum',
 'bulmak',
 'bay',
 'yap',
 'arkadaşın',
 'sevindim',
 'sürü',
 'temiz',
 'baksana',
 'inanamıyorum',
 'sıkı',
 'düşünüyorsun',
 'hepiniz',
 's',
 'aferin',
 'tuhaf',
 'eminim',
 'yapacak',
 'yüzbaşı',
 'pardon',
 'gitmem',
 'ateş',
 'gurur',
 'general',
 'yolunda',
 'pislik',
 'zamanı',
 'tanrının',
 'bayan',
 'bayanlar',
 'sence',
 'buldun',
 'yapacaksın',
 'silah',
 'frank',
 'başkan',
 'tür',
 'the',
 'bayım',
 'ettin',
 'bilir',
 'saçma',
 'güvenli',
 'bağlı',
 'sefer',
 'ahbap',
 'görünüyor',
 'mükemmel',
 'bekliyor',
 'kahretsin',
 'üzgünüm',
 'silahı',
 'kapa',
 'olmalısın',
 'olası',
 'olmalı',
 'parçası',
 'olduğumu',
 'umarım',
 'çeneni',
 'güvenlik',
 'alır',
 'herşey',
 'film',
 'sanmıyorum',
 'şeyin',
 'dedektif',
 'sebep',
 'bahsediyorsun',
 'anlamı',
 'değiliz',
 'görünüşe',
 'güç',
 'dolar'

In [33]:
len(not_in_sent_selected_word_list)

133

In [34]:
df_youtube_sent_select.drop(["word_index_list"], axis=1, inplace=True)
df_youtube_sent_select

/home/kurubal/anaconda3/envs/py39/lib/python3.9/site-packages/pandas/core/frame.py:4906: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().drop(


,search_string,start_time,end_time,sentence,video_id,video_url
0,başka biriyle herhangi bir şey yaşıyor musun k...,319,325,başka biriyle herhangi bir şey yaşıyor musun k...,KsZ_8oIQsv4,https://www.youtube.com/watch?v=KsZ_8oIQsv4&t=...
1,sıcak mı evet böyle işte tamam bir şey yok ora...,603,612,sıcak mı evet böyle işte tamam bir şey yok ora...,Mh3bgDLKt2o,https://www.youtube.com/watch?v=Mh3bgDLKt2o&t=...
2,benim burada ne işim var neden bunu yapıyorum ...,273,278,benim burada ne işim var neden bunu yapıyorum ...,sHnvEDmE3so,https://www.youtube.com/watch?v=sHnvEDmE3so&t=...
3,bırakın falan diyor tamam tahmin ettim zaten ç...,350,356,uygulamayı bırakın falan diyor tamam tahmin et...,cmQjfzOKFII,https://www.youtube.com/watch?v=cmQjfzOKFII&t=...
4,belli değil benim ne yapacağım belli değil ne ...,93,99,kaderim belli değil benim ne yapacağım belli d...,VBdcD7hLGuo,https://www.youtube.com/watch?v=VBdcD7hLGuo&t=93s
...,...,...,...,...,...,...
181752,ol o o ne,633,636,abone ol o o ne arabası bu arkadaşlar zaten ka...,0Y065FGH_T0,NaN
181753,ol o o i,606,609,abone ol o o i yi akşamlar bilmediğim için abo...,JyzAOR0GTs4,NaN
181754,mi o o o,419,428,mi o o o,5ofdGcvaeZU,NaN
181755,a a i ya,602,603,a a i ya,ZijHforW6q4,NaN


In [35]:
def word_in_wordgroup(source_word_list, df_target, target_column, word_sample_num, simple=False):

    '''word_in_wordgroup(not_in_sent_word_list, df_youtube_sent_select, "search_string", 5, simple=False)\n
       source_word_list is searching word list\n
       df_target is dataframe, target_column are dataframe column string name\n
       word_sample_num is searching sample number.
       simple use for all column result or only target column result 
    '''
    if simple:
        df_select = df_target[[f"{target_column}"]].dropna()
    else:
        df_select = df_target
        
    df_result = pd.DataFrame()
    for i in source_word_list:
        try:
            word_in_word_cluster = df_select[df_select[f"{target_column}"].str.contains(fr"(?:\s|^){i}(?:\s|$)", na=True)].head(word_sample_num)    
        except:
            pass        
        word_in_word_cluster.insert(0,"word",i)
        df_result = pd.concat([df_result,word_in_word_cluster], axis=0)
    df_result.reset_index(drop=True, inplace=True)

    return df_result

In [36]:
if youtube_sent_usage == "all": 
    df_youtube_usage_select = df_youtube_sent_result
else:
    df_youtube_usage_select = df_youtube_sent_select

In [37]:
not_in_sent_selected_word_list_result = word_in_wordgroup(not_in_sent_selected_word_list, df_youtube_usage_select, "search_string", 5, simple=False)
not_in_sent_selected_word_list_result

,word,search_string,start_time,end_time,sentence,video_id,video_url
0,fikrim,diyorum ama aslında bununla ilgili hiçbir fikr...,33,39,diyorum ama aslında bununla ilgili hiçbir fikr...,4syqfwNfygw,https://www.youtube.com/watch?v=4syqfwNfygw&t=33s
1,fikrim,var içinde ne olduğu hakkında hiçbir fikrim yo...,41,46,şöyle bir dolap var içinde ne olduğu hakkında ...,3sSYXQnu6AU,https://www.youtube.com/watch?v=3sSYXQnu6AU&t=41s
2,fikrim,harika bir müzik fikrim var ve babam bize yard...,542,546,şarkı sözlerine zaten sahibiz sadece diğer her...,VyFnAuumYKk,https://www.youtube.com/watch?v=VyFnAuumYKk&t=...
3,fikrim,nasıl olduğu konusunda hiçbir fikrim yok ama ş...,58,61,işığın nasıl olduğu konusunda hiçbir fikrim yo...,ozaCwmUvl4A,https://www.youtube.com/watch?v=ozaCwmUvl4A&t=58s
4,fikrim,neden böyle olduğu hakkında hiçbir fikrim yok ...,29,32,genelde öncesinde deneyimliyorum neden böyle o...,7L64XN3or3I,https://www.youtube.com/watch?v=7L64XN3or3I&t=29s
...,...,...,...,...,...,...,...
614,tıpkı,şeyler sana geliyor tıpkı benim burada bugün,651,654,ve hayal bile edemeyeceğin şeyler sana geliyor...,coSkDSz35Sg,https://www.youtube.com/watch?v=coSkDSz35Sg&t=...
615,tıpkı,tıpkı senin gibi ben de bir şeyler hayal,184,186,capcanlı karşınızda duruyor tıpkı senin gibi b...,yxE4H6s7ULM,https://www.youtube.com/watch?v=yxE4H6s7ULM&t=...
616,tıpkı,da tıpkı insanlar gibi bir hayat yaşıyor,14,17,markalar da tıpkı insanlar gibi bir hayat yaşıyor,FL0CMvwXPlM,https://www.youtube.com/watch?v=FL0CMvwXPlM&t=14s
617,tıpkı,tıpkı senin ile kızın arasında ki güçlü,31,35,tıpkı senin ile kızın arasında ki güçlü bağ gibi,ezd_IKyHM_A,https://www.youtube.com/watch?v=ezd_IKyHM_A&t=31s


In [38]:
#not_in_sent_selected_word_list_merge = pd.merge(not_in_sent_selected_word_list_result, df_youtube_sent_select, how="left", on="search_string")
#not_in_sent_selected_word_list_merge.drop_duplicates(inplace=True)
#not_in_sent_selected_word_list_merge

In [38]:
not_in_sent_selected_word_list_result.word.nunique()

126

In [39]:
set_not_in_sent_selected_word_list = set(not_in_sent_selected_word_list)
set_not_in_sent_selected_word_list_result = set(not_in_sent_selected_word_list_result["word"].to_list())

In [40]:
set_not_in_sent_selected_word_list.difference(set_not_in_sent_selected_word_list_result)

{'ahbap', 'hl', 'kahrolası', 'orospu', 'pekl', 'seks', 'yüzbaşı'}

In [41]:
#word_in_wordgroup_simple(["pekala","gece"], df_youtube_sent_select, "search_string", 5)

In [42]:
not_in_sent_selected_word_list_result.to_excel(f"{lang_folder.capitalize()}_{word_end}_Word_Group_In_Youtube_Sentence_Word_Index_List_Sample2.xlsx", index=False)

#### Copy Move And Delete

In [43]:
output_file = glob.glob(f"{lang_folder.capitalize()}_{word_end}_Word_Group_In_Youtube_Sentence_*.xlsx")
output_file

['Turkish_1000_Word_Group_In_Youtube_Sentence_Word_Index_List_Sample.xlsx',
 'Turkish_1000_Word_Group_In_Youtube_Sentence_Word_Index_Word_Count.xlsx',
 'Turkish_1000_Word_Group_In_Youtube_Sentence_Word_Index_List_Sample2.xlsx']

In [44]:
for k in output_file:
    source = k # source directory
    destination = path
    shutil.copy2(source, destination)

In [45]:
for i in output_file:
    try:
        os.remove(i)
    except:
        pass